## 집계연산과 조인

#### 지시사항

* 데이터 읽어오기 : data/trip-data 경로 아래에 있는 201508_station_data.csv, 201508_trip_data.csv 를 읽으세요. header=True 옵션을 주어야 합니다.

* station 데이터를 활용하여, landmark별 station_id의 개수(중복제거)를 추출하세요. (group by,countDistinct)

* trip 데이터를 활용하여, ‘Subscriber Type’별 ‘Bike #’ 합, ‘Duration’ 평균, ‘Trip ID’ 수를 구하세요.
(group by,sum,avg,count)

* trip 데이터의 ‘Start Station’ 컬럼을 활용해서, ‘name’ 컬럼을 만들어주세요. 생성한 ‘name’ 컬럼과 station 데이터의 ‘name’ 컬럼을 키로 inner join , left anti을 각각 수행하세요. 마지막으로 select로 ‘name’을 선택하여 중복제거한 카운트가 얼마인지 구하세요.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# csv 파일 읽기
trip = spark.read.csv("data/trip-data/201508_trip_data.csv", header=True)
station = spark.read.csv("data/trip-data/201508_station_data.csv", header=True)

trip.show()
station.show()

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|       Start Station|Start Terminal|       End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

In [4]:
# landmark 집계 
landmark = station.groupby('landmark').agg(countDistinct('station_id'))
landmark.show()

+-------------+--------------------------+
|     landmark|count(DISTINCT station_id)|
+-------------+--------------------------+
|    Palo Alto|                         5|
|San Francisco|                        35|
|     San Jose|                        16|
| Redwood City|                         7|
|Mountain View|                         7|
+-------------+--------------------------+



In [5]:
# Subscriber Type 집계
subscriber = trip.groupby('Subscriber Type').agg(sum('Bike #'), avg('Duration'), count('Trip ID'))
subscriber.show()

+---------------+------------+-----------------+--------------+
|Subscriber Type| sum(Bike #)|    avg(Duration)|count(Trip ID)|
+---------------+------------+-----------------+--------------+
|     Subscriber|1.31370297E8|597.2995870632493|        310217|
|       Customer| 1.8421093E7|4214.454831000341|         43935|
+---------------+------------+-----------------+--------------+



In [6]:
# 이너조인
df_inner = trip.withColumn('name', col('Start Station')).join(station, on='name', how = 'inner').select(countDistinct('name'))
df_inner.show()

+--------------------+
|count(DISTINCT name)|
+--------------------+
|                  68|
+--------------------+



In [10]:
# 안티조인
df_anti = trip.withColumn('name', col('Start Station')).join(station, on='name', how = 'left_anti').select(countDistinct('name'))
df_anti.show()

+--------------------+
|count(DISTINCT name)|
+--------------------+
|                   2|
+--------------------+

